Step 3. A bit... "cleaner".

Cleans up known code and comments. Breaks out LaTeX.

In [ ]:
import os 
try:
    from os import scandir, walk
except ImportError:
    from scandir import scandir, walk
from nbformat import v4 as nbf
import re
def get_m_files(d="."):
    exts=["m"]
    for root, dirs, files in walk(d, topdown=False):
        for name in files:
            if os.path.splitext(name)[1].strip('.') in exts:
                yield os.path.abspath(os.path.join(root, name))

In [31]:
class matlab_to_python_line_transmogrification(object):
    HEADER=0
    COMMENT=1
    CODE=2
    MARKDOWN=3
    """ trans·mog·ri·fied
        trans·mog·ri·fy·ing
        transitive verb
        to change or alter greatly and often with grotesque or humorous effect
    """
    def __init__(self, line):
        self.line = line
        # print("B. "+line)
        if line.startswith("%%"):
            self.type=matlab_to_python_line_transmogrification.HEADER
            self.cell_type = matlab_to_python_line_transmogrification.MARKDOWN
        elif line.startswith("%"):
            self.type=matlab_to_python_line_transmogrification.COMMENT
            self.cell_type = matlab_to_python_line_transmogrification.MARKDOWN
        else:
            self.type=matlab_to_python_line_transmogrification.CODE
            self.cell_type = matlab_to_python_line_transmogrification.CODE
            
    def __repr__(self):
        return "mtplt({})".format(self.line)

    def __str__(self):
        line=self.line.strip()
        line=line.strip("% ")
        if len(line)==0:
            return ""
        if self.type == matlab_to_python_line_transmogrification.HEADER:    
            return "\n\n# {}\n\n".format(line)
        if self.type == matlab_to_python_line_transmogrification.COMMENT:          
            badwords=["Key MATLAB commands", "www.mathworks.com"]
            for badword in badwords:
                if badword in line:
                    return ""
            if line.startswith("* "):
                line="\n"+line
            prefix=""
            postfix=""
            return "{}{}{}".format(prefix,line,postfix)
        if self.type == matlab_to_python_line_transmogrification.CODE:
            line=self.line
            # print("C. "+line)
            return "{}".format(line)
        return ""
    
def cell_cleaner_upperer(notebook_cell):
    source = notebook_cell["source"]
    if notebook_cell["cell_type"] is "markdown":
        # Non inline latex
        source=re.sub(r"\$\$([^$]+)\$\$",r"\n\n$$\n\1\n$$\n\n",source)
        # Embedded Functions
        source=re.sub(r"\|([\w_]+)\|",r"```\1```",source)
        # Links to other
        source=re.sub(r"< \?example=([\w]+)&section=([\w]+) ([^>]+)>",
                      r"[\3](/examples/\1/\1-\2.ipynb)",source)
    if notebook_cell["cell_type"] is "code":
        cleaned_lines=list()
        lines = source.split("\n")
        for line in lines:
            try:
                exec(line)
            except:
                line = "# " + line
            cleaned_lines.append(line)
        source="\n".join(cleaned_lines)
    #print(source, end="")
    notebook_cell["source"] = source
    return notebook_cell

def convert_m_script_to_ipython_notebook(m_file):
    nb = nbf.new_notebook()
    
    example, section = os.path.splitext(os.path.basename(m_file))[0].split("_")
    title = "{} - {}".format(re.sub(r"(\w)([A-Z])", r"\1 \2", example),
                             re.sub(r"(\w)([A-Z])", r"\1 \2", section))
    slug = "{}-{}".format(re.sub(r"(\w)([A-Z])", r"\1_\2", example.lower()),
                             re.sub(r"(\w)([A-Z])", r"\1_\2", section.lower()))
    with open (m_file, "r") as fid:
        lines=fid.readlines()
    
    last_mtplt=None
    for line in lines:
        mtplt = matlab_to_python_line_transmogrification(line)
        if last_mtplt is None:
            running_block=""           
            notebook_cells=list()
        else:
            if mtplt.cell_type != last_mtplt.cell_type:
                if last_mtplt.cell_type == 3:
                    cell_tmp = nbf.new_markdown_cell(running_block.strip())
                if last_mtplt.cell_type == 2:
                    cell_tmp = nbf.new_code_cell(running_block.strip())
                notebook_cells.append(cell_tmp)
                running_block=str_tmp=str(mtplt)
                last_mtplt = mtplt
                continue
        str_tmp=str(mtplt)
        if len(str_tmp) == 0:
            continue
        if running_block.endswith("\n"):
            running_block=(running_block+str_tmp)
        else:
            running_block=(running_block+" "+str_tmp)
        last_mtplt = mtplt
        

        
    ## Start Cells
    nb["cells"].append(nbf.new_markdown_cell(r"""<div id="toc"></div>"""))
        
    ## Body cells
    for notebook_cell in notebook_cells:
        cleaned_cell=cell_cleaner_upperer(notebook_cell)
        nb["cells"].append(cleaned_cell)
        
    ## End Cells
    nb["cells"].append(nbf.new_code_cell(r"""%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')"""))

    nikola_metadata=dict()
    nikola_metadatas=dict()
    nikola_metadata["author"]=""
    nikola_metadata["category"]=example
    nikola_metadata["date"]=""
    nikola_metadata["description"]=""
    nikola_metadata["link"]=""
    nikola_metadata["slug"]=slug
    nikola_metadata["tags"]=section
    nikola_metadata["title"]=title
    nikola_metadata["type"]=""
    nikola_metadatas["nikola"]=nikola_metadata
    nb["metadata"]=nikola_metadatas
    return nb

In [32]:
pp = ExecutePreprocessor()
pp.timeout = 30  # seconds
pp.interrupt_on_timeout = True

for m in get_m_files():
    ipynb = os.path.splitext(m)[0]+"_03.ipynb"
    nb = convert_m_script_to_ipython_notebook(m)
    nb_executed, resources = pp.preprocess(nb, resources={})
    with open(ipynb, 'w') as f:
        print(nbf.writes(nb_executed), file=f)